In [9]:
import os
import shutil
from os.path import join
import glob
Folder = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/AFDB_face_dataset/*/*'
Folder1 = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/AFDB_masked_face_dataset/*/*'
des ='/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/data/unmasked/' 
des1 = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/data/mask/' 
for img in glob.glob(Folder1):
    shutil.copy(img, des1)
masklen = len(os.listdir(des1))
print(masklen)
for img in glob.glob(Folder):
    shutil.copy(img, des)
    unmasklen = len(os.listdir(des))
    if unmasklen >= masklen:
        break


In [9]:
train_dir = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train' #path for train dir
validation_dir = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/test' # path for val dir

train_horses_dir = os.path.join(train_dir, 'unmasked') #instance for label-1 of train
train_humans_dir = os.path.join(train_dir, 'mask') #instance for label-2 of train
validation_horses_dir = os.path.join(validation_dir, 'unmasked') #instance for label-1 of val
validation_humans_dir = os.path.join(validation_dir, 'mask') #instance for label-2 of val


In [10]:
import random
from shutil import copyfile
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    all_files = []
    
    for file_name in os.listdir(SOURCE):
        file_path = SOURCE + file_name

        if os.path.getsize(file_path):
            all_files.append(file_name)
        else:
            print('{} is zero length, so ignoring'.format(file_name))
    
    n_files = len(all_files)
    split_point = int(n_files * SPLIT_SIZE)
    
    shuffled = random.sample(all_files, n_files)
    
    train_set = shuffled[:split_point]
    test_set = shuffled[split_point:]
    
    for file_name in train_set:
        copyfile(SOURCE + file_name, TRAINING + file_name)
        
    for file_name in test_set:
        copyfile(SOURCE + file_name, TESTING + file_name)


unmasked_SOURCE_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/data/unmasked/"
TRAINING_unmasked_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train/unmasked/"
TESTING_unmasked_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/test/unmasked/"
mask_SOURCE_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/data/mask/"
TRAINING_mask_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train/mask/"
TESTING_mask_DIR = "/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/test/mask/"

split_size = .6
split_data(unmasked_SOURCE_DIR, TRAINING_unmasked_DIR, TESTING_unmasked_DIR, split_size)
split_data(mask_SOURCE_DIR, TRAINING_mask_DIR, TESTING_mask_DIR, split_size)


In [11]:
# train_dir = '/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train/unmasked'
# train_unmasked_dir = os.path.join(train_dir, 'unmasked')
# train_mask_dir = os.path.join(train_dir, 'mask')
train_unmasked_fnames = os.listdir('/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train/unmasked')
train_unmasked_fnames = ['unmasked' for x in train_unmasked_fnames]
train_mask_fnames = os.listdir('/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/train/mask')
train_mask_fnames = ['mask' for x in train_mask_fnames]
test_unmasked_fnames = os.listdir('/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/test/unmasked')
test_unmasked_fnames = ['unmasked' for x in test_unmasked_fnames]
test_mask_fnames = os.listdir('/Users/nikhi/Desktop/cps/RMFD/self-built-masked-face-recognition-dataset/test/mask')
test_mask_fnames = ['mask' for x in test_mask_fnames]
print(len(train_unmasked_fnames))
print(len(train_mask_fnames))
print(len(test_unmasked_fnames))
print(len(test_mask_fnames))

1003
1006
566
569


In [16]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model = load_model('maskdetectionmodel.h5')

In [14]:
model1 = load_model('simplemaskdetectionmodel.h5')

In [ ]:
import numpy as np
import cv2
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict={1:'without_mask',0:'with_mask'}
color_dict={1:(0,0,255),0:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        result = int(np.where(result > 0.5, 1, 0))
        #print(result)
        
        label= result
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()